## Add path to source files

In [2]:
import sys, os
sys.path.insert(0, os.path.join(os.path.abspath('..'), 'src'))

## Import necessary packages, add loading bars, make plots big

In [3]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import FiniteDifference as fd
import LiquidCrystalHelper as lch
import biharm as bh

import time
from importlib import reload

In [4]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Video
from IPython.display import Image

In [5]:
reload(lch)

<module 'LiquidCrystalHelper' from 'C:\\Users\\lucas\\Documents\\Grad Work\\Summer Research 2020\\LiquidCrystalHydrodynamics\\src\\LiquidCrystalHelper.py'>

In [6]:
%matplotlib qt
dpi = 300
mpl.rcParams['figure.dpi'] = dpi

## Begin with a $257\times 257$ grid, with one $+1/2$ disclination and one $-1/2$ disclination
* The reason for this odd choice of size is that the FFT works best on an array of size $N = 2^i - 1$ (and the same for the other dimension).
* In this algorithm, the thing that is being Fourier Transformed is the interior points of our domain. This will be $255\times 255$ since the endpoints in either dimension are cutoff.
* Clearly this is $(2^i - 1)\times(2^i - 1)$ for $i = 8$. 

### Set up domain

In [27]:
l = 140
n = 257

x = np.linspace(-l, l, num=n)
y = np.linspace(-l, l, num=n)
X, Y = np.meshgrid(x, y, indexing='ij')

### Create two disclinations, get auxiliary variables

In [28]:
S_val = 0.5
ctr = [[-35, 0], [35, 0]]
r = [1, 1]
m = [-1/2, 1/2]

S, phi = lch.makeMultiDisclination(X, Y, S_val, ctr, r, m)
Q = lch.uniaxialQ(S, phi)
eta, mu, nu = lch.auxVars(Q)

### Get initial eigenvectors/eigenvalues, plot

In [31]:
sparse_shape = (10, 10)
S_sparse_shape = (200, 200)
S_cutoff = 0.3

lambda_max = lch.calcQEigenvals(eta, mu, nu)
U, V = lch.calcQEigenvecs(eta, mu, nu, lambda_max, S_cutoff)

sparse_idx = lch.sparseIdx(Q[0, 0].shape, sparse_shape)
S_sparse_idx = lch.sparseIdx(Q[0, 0].shape, S_sparse_shape)

fig, ax = plt.subplots(figsize=(4, 3))
c = ax.pcolor(X[S_sparse_idx], Y[S_sparse_idx], (3/2)*lambda_max[S_sparse_idx], vmin=0, vmax=1.5)
q = ax.quiver(X[sparse_idx], Y[sparse_idx], U[sparse_idx], V[sparse_idx],
              headwidth=0, pivot='middle', headaxislength=5, scale=30, width=0.002)

ax.axis('equal')
fig.colorbar(c, ax=ax)
ax.set_title("+1/2 (left) and -1/2 (right) defects")
ax.set_xlabel(r"$x/\xi$")
ax.set_ylabel(r"$y/\xi$")

Text(0, 0.5, '$y/\\xi$')

### First, see how long it takes the configuration to relax

In [44]:
dx = X[1, 0] - X[0, 0]
dy = Y[0, 1] - Y[0, 0]

# This is just what they said it should be for stability
dt = dx**2/16

num_steps = 50
num_peaks = 2
peak_pos = np.zeros((num_peaks, num_steps))
peak_val = np.zeros((num_peaks, num_steps))
t = np.zeros(num_steps)

eta_old = eta
mu_old = mu
nu_old = nu

# Create loading bar
progress = widgets.IntProgress(min=0, max=num_steps - 1)
display(progress)

start_time = time.perf_counter()
for i in range(num_steps):
    
    eta_new = fd.forwardEuler(eta_old, dt, lch.etaEOM, mu_old, nu_old, dx, dy)
    mu_new = fd.forwardEuler(mu_old, dt, lch.muEOM, eta_old, nu_old, dx, dy)
    nu_new = fd.forwardEuler(nu_old, dt, lch.nuEOM, eta_old, mu_old, dx, dy)
    
    eta_old = eta_new
    mu_old = mu_new
    nu_old = nu_new
    
    lambda_max = lch.calcQEigenvals(eta_old, mu_old, nu_old)
    peaks = lch.findMinima(lambda_max)
    if len(peaks[0]) <= 2:
        peak_pos[:, i] = X[peaks]
        peak_val[:, i] = lambda_max[peaks]
    else:
        break
        
    t[i] = i*dt
    
    # Update loading bar
    progress.value = i
    
# truncate arrays if loop stops early
t = t[:i]
peak_pos = peak_pos[:, :i]
peak_val = peak_val[:, :i]

eta_hydro = eta_new
mu_hydro = mu_new
nu_hydro = nu_new
    
end_time = time.perf_counter()
print("Simulation ran in", (end_time - start_time)/60, "minutes.")

IntProgress(value=0, max=49)

Simulation ran in 0.011166271666667171 minutes.


In [45]:
plt.plot(t, peak_val[0, :])
plt.title(r'$\lambda_\mathrm{max}$ vs. $t$ for $dt = $' + str(round(dt, 5)))
plt.xlabel(r'$t/\tau$')
plt.ylabel(r'$\lambda_\mathrm{max}$')

Text(0, 0.5, '$\\lambda_\\mathrm{max}$')

### Plot the configuration at this point

In [46]:
sparse_shape = (20, 20)
S_sparse_shape = (200, 200)
S_cutoff = 0.3

lambda_max = lch.calcQEigenvals(eta_old, mu_old, nu_old)
U, V = lch.calcQEigenvecs(eta_old, mu_old, nu_old, lambda_max, S_cutoff)

sparse_idx = lch.sparseIdx(Q[0, 0].shape, sparse_shape)
S_sparse_idx = lch.sparseIdx(Q[0, 0].shape, S_sparse_shape)

fig, ax = plt.subplots(figsize=(5, 3))
c = ax.pcolor(X[S_sparse_idx], Y[S_sparse_idx], (3/2)*lambda_max[S_sparse_idx], vmin=0, vmax=1.5)
q = ax.quiver(X[sparse_idx], Y[sparse_idx], U[sparse_idx], V[sparse_idx],
              headwidth=0, pivot='middle', headaxislength=5, scale=30, width=0.002)

ax.axis('equal')
fig.colorbar(c, ax=ax)
ax.set_title("+1/2 (left) and -1/2 (right) defects")
ax.set_xlabel(r"$x/\xi$")
ax.set_ylabel(r"$y/\xi$")

Text(0, 0.5, '$y/\\xi$')

### Set up biharmonic solver

In [47]:
L = [2*l, 2*l]
shape = [n - 1, n - 1]
alpha = lch.alpha
maxiter = 500

bh_solver = bh.Biharm(L, shape, alpha, cg_maxiter=maxiter)

### Now add in the hydrodynamic effects and run for a couple of steps to make sure it doesn't fall apart

In [76]:
dt = dx**2/16

num_steps = 1000
num_peaks = 2
peak_pos = np.zeros((num_peaks, num_steps))
peak_val = np.zeros((num_peaks, num_steps))
t = np.zeros(num_steps)

eta_old = eta_hydro
mu_old = mu_hydro
nu_old = nu_hydro

kernel = -np.ones((5, 5))
kernel[2, 2] = 24

lambda_max_ar = np.zeros(eta_old.shape + (num_steps,))
psi = np.zeros(eta_old.shape)

# Create loading bar
progress = widgets.IntProgress(min=0, max=num_steps - 1)
display(progress)

start_time = time.perf_counter()
for i in range(num_steps):
    
    source_term = ( lch.beta*lch.f1(eta_old, mu_old, nu_old, dx) 
                    + (1/4)*lch.alpha*lch.f2(eta_old, mu_old, nu_old, dx) )
    psi[1:-1, 1:-1], info, calls = bh_solver.solve(source_term[1:-1, 1:-1])
    
    eta_new = fd.forwardEuler(eta_old, dt, lch.etaFlowEOM, mu_old, nu_old, psi, dx)
    mu_new = fd.forwardEuler(mu_old, dt, lch.muFlowEOM, eta_old, nu_old, psi, dx)
    nu_new = fd.forwardEuler(nu_old, dt, lch.nuEOM, eta_old, mu_old, dx)
    
    eta_old = eta_new
    mu_old = mu_new
    nu_old = nu_new
    
    lambda_max = lch.calcQEigenvals(eta_old, mu_old, nu_old)
    lambda_max_ar[:, :, i] = lambda_max
    
#     peaks = lch.findMinima(lambda_max)
#     if len(peaks[0]) <= num_peaks:
#         peak_pos[:, i] = X[peaks]
#         peak_val[:, i] = lambda_max[peaks]
#     else:
#         break
        
    t[i] = i*dt
    
    # Update loading bar
    progress.value = i
    
#     plt.matshow(lambda_max, vmin=-1, vmax=1)
#     plt.colorbar()
    
# truncate arrays if loop stops early
t = t[:i]
peak_pos = peak_pos[:, :i]
peak_val = peak_val[:, :i]
    
end_time = time.perf_counter()
print("Simulation ran in", (end_time - start_time)/60, "minutes.")

IntProgress(value=0, max=999)

Simulation ran in 6.735762468333333 minutes.


### Make flow plot for the stream function

In [77]:
vx, vy = fd.curl(psi, dx)
v = np.sqrt(vx**2 + vy**2)

mask = np.where(v != 0)
vx_norm = np.zeros(vx.shape)
vy_norm = np.zeros(vy.shape)
vx_norm[mask] = vx[mask]/v[mask]
vy_norm[mask] = vy[mask]/v[mask]

sparse_idx = lch.sparseIdx(vx.shape, (30, 30))

fig, ax = plt.subplots()
c = ax.pcolor(X, Y, v)
fig.colorbar(c, ax=ax)
q = ax.quiver(X[sparse_idx], Y[sparse_idx], 
              vx_norm[sparse_idx], vy_norm[sparse_idx], pivot='tail', color='w')
ax.plot(X[peaks], Y[peaks], marker='o', ls='', c='r', label='Defects')
ax.legend()
ax.set_title('Flow fields at introduction of hydrodynamics')
ax.set_xlabel(r'$x/\xi$')
ax.set_ylabel(r'$y/\xi$')

Text(0, 0.5, '$y/\\xi$')

In [78]:
fig, ax = plt.subplots()
im = ax.imshow(lambda_max_ar[:, :, -1], vmin=-1, vmax=1, origin='lower')
im.set_data(lambda_max_ar[:, :, 0])
# plt.plot(peaks[1], peaks[0], marker='o', ls='')
fig.colorbar(im, ax=ax)

In [80]:
t[-1]

74.6185302734375

### Make an animation of the eigenvalues getting out of control

In [ ]:
fig, ax = plt.subplots()
im = ax.matshow(lambda_max_ar[:, :, 30], vmin=-1, vmax=1)
fig.colorbar(mpl.cm.ScalarMappable())
plt.title(r"$\lambda_\mathrm{max}$")
plt.xlabel(r"$x/\xi$")
plt.ylabel(r"$y/\xi$")

def init():
    im.set_data(lambda_max_ar[:, :, 30])
    return im,

def update(frame):
    if frame <= num_steps - 1:
        im.set_data(lambda_max_ar[:, :, frame])
    return im,

ani = FuncAnimation(fig, update, frames=np.arange(30, num_steps + 4),
                    init_func=init, interval=500, blit=False)

In [ ]:
ani.save("FlowCatastrophe.mp4", dpi=200)

In [ ]:
%matplotlib inline

In [ ]:
plt.matshow(psi, vmin=-1, vmax=1, origin='lower')
plt.colorbar()

### Let's just try a convolution

In [ ]:
from scipy.signal import convolve2d

In [ ]:
help(convolve2d)

In [ ]:
kernel1 = -np.ones((3, 3))
kernel1[1, 1] = 8
kernel2 = -np.ones((5, 5))
kernel2[2, 2] = 24

conv = convolve2d(lambda_max, kernel2, mode='valid')
plt.matshow(conv)
plt.colorbar()

In [ ]:
plt.matshow(lambda_max)
plt.colorbar()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot([], [], 'ro')

def init():
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1, 1)
    return ln,

def update(frame):
    xdata.append(frame)
    ydata.append(np.sin(frame))
    ln.set_data(xdata, ydata)
    return ln,

ani = FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128),
                    init_func=init, interval=500, blit=True)
plt.show()

In [ ]:
ani.save("Example.mp4", dpi=200)